In [1]:
## Setup
from __future__ import print_function
import time, os, json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import csv

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

def get_session():
    """Create a session that dynamically allocates memory."""
    # See: https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)
    return session

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2


In [2]:
BATCH_SIZE = 64
NUM_CLASSES = 16
TRAIN_RATIO = 0.6

import utilities
# Datafile options:
#imu_data,x_data,y_norm = utilities.load_data('/media/Big_Data/stefantj/flows_lowres_16k.npy')
#imu_data,flows_data,y_norm = utilities.load_data('../gbucket/center_cropped_300x300.npy')
#imu_data,x_data,y_norm = utilities.load_data('../gbucket/center_cropped_192x192.npy')
imu_data,x_data,y_norm = utilities.load_data(['flows_lowres_1_16.npy','flows_lowres_2_16.npy','flows_lowres_3_16.npy','flows_lowres_4_16.npy','flows_lowres_5_16.npy','flows_lowres_6_16.npy','flows_lowres_7_16.npy','flows_lowres_8_16.npy','flows_lowres_9_16.npy','flows_lowres_10_16.npy','flows_lowres_11_16.npy','flows_lowres_12_16.npy','flows_lowres_13_16.npy','flows_lowres_14_16.npy','flows_lowres_15_16.npy','flows_lowres_16_16.npy'])# Split into training/val data:
N_training = int(TRAIN_RATIO*x_data.shape[0])
N_validation = x_data.shape[0]- N_training

y_data, mean_vals = utilities.discretize_outputs(y_norm[0:N_training], NUM_CLASSES)
y_data, mean_vals = utilities.discretize_outputs(y_norm, NUM_CLASSES)

x_train = x_data[0:N_training,...]
y_train = y_data[0:N_training]

x_val = x_data[N_training:,...]
y_val = y_data[N_training:]


Loading vector data
Loading image data
flows_lowres_1_16.npy
flows_lowres_2_16.npy
flows_lowres_3_16.npy
flows_lowres_4_16.npy
flows_lowres_5_16.npy
flows_lowres_6_16.npy
flows_lowres_7_16.npy
flows_lowres_8_16.npy
flows_lowres_9_16.npy
flows_lowres_10_16.npy
flows_lowres_11_16.npy
flows_lowres_12_16.npy
flows_lowres_13_16.npy
flows_lowres_14_16.npy
flows_lowres_15_16.npy
flows_lowres_16_16.npy
Preprocessing data


In [3]:
BATCH_SIZE = 256
NUM_CLASSES = 16
from classifiers.resnet import *

hps = HParams(batch_size=BATCH_SIZE,
                         num_classes=NUM_CLASSES,
                         min_lrn_rate=0.1,
                         lrn_rate=.01,
                         num_residual_units=5,
                         use_bottleneck=False,
                         weight_decay_rate=0.0002,
                         relu_leakiness=0.1,
                         optimizer='sgd')

batch_x, batch_y = utilities.sample_subseq(BATCH_SIZE, x_train, y_train)
# Need to reshape 
batch_yhot= np.zeros((BATCH_SIZE, NUM_CLASSES));
batch_yhot[np.arange(BATCH_SIZE), batch_y.astype(np.int32)] = 1
LR = 0.1
with tf.device('/cpu:0'):
#    sess = get_session()
#    model = ResNet(hps, batch_x, batch_yhot, 'train')
#    model.build_graph()
#    optimizer = tf.train.AdamOptimizer(learning_rate=LR).minimize(model.cost)

#    init = tf.variables_initializer(tf.global_variables())
#    sess.run(init)

#    saver = tf.train.Saver()
    saver.restore('.')
    N_steps = 1000
    costs = []
    for k in range(N_steps):
        batch_x, batch_y = utilities.sample_subseq(BATCH_SIZE, x_train, y_train)
        # Need to reshape 
        batch_yhot= np.zeros((BATCH_SIZE, NUM_CLASSES));
        batch_yhot[np.arange(BATCH_SIZE), batch_y.astype(np.int32)] = 1

        o, lr = sess.run([optimizer,model.lrn_rate], feed_dict={model._images:batch_x, model.labels:batch_yhot})

        cost, pred = sess.run([model.cost, model.predictions], feed_dict={model._images:batch_x, model.labels:batch_yhot})
        costs.append(cost)
        print('Step: ', k, ' Cost: ', cost, ' LR: ', lr)
        print(pred[0])
        if(k % 100 is 0 and k > 0): 
            plt.plot(costs)
            plt.show()
            saver.save(sess, 'resnet_model', global_step = k)        

Step:  0  Cost:  3.72909  LR:  0.01
[ 0.02042483  0.14770205  0.05017251  0.00937276  0.02092651  0.03488031
  0.04333722  0.02505529  0.08860323  0.01205038  0.01833478  0.01584185
  0.16855596  0.07024016  0.172185    0.10231724]
Step:  1  Cost:  4.30366  LR:  0.01
[ 0.05193567  0.16155492  0.1078655   0.01287588  0.06115754  0.07828233
  0.0590331   0.10979384  0.04883147  0.02322232  0.02898748  0.01318973
  0.05658418  0.05799892  0.11407801  0.01460909]
Step:  2  Cost:  4.88092  LR:  0.01
[ 0.07561139  0.07427808  0.07568002  0.03292467  0.07036099  0.06538712
  0.15266795  0.06164448  0.03797018  0.05249106  0.06489255  0.03470464
  0.05149877  0.0630962   0.0525758   0.03421609]
Step:  3  Cost:  5.54844  LR:  0.01
[ 0.06543647  0.05937027  0.04950536  0.06606957  0.05468657  0.04913067
  0.07952889  0.03710287  0.0499747   0.07854971  0.07977046  0.05566016
  0.06430607  0.07265385  0.07351085  0.06474351]
Step:  4  Cost:  6.04989  LR:  0.01
[ 0.04560798  0.01791673  0.32442349

KeyboardInterrupt: 

In [ ]:
with tf.device('/cpu:0'):
    N_steps = 1000
    costs = []
    for k in range(N_steps):
        batch_x, batch_y = utilities.sample_subseq(BATCH_SIZE, x_train, y_train)
        # Need to reshape 
        batch_yhot= np.zeros((BATCH_SIZE, NUM_CLASSES));
        batch_yhot[np.arange(BATCH_SIZE), batch_y.astype(np.int32)] = 1

        o, lr = sess.run([model.train_op, model.lrn_rate], feed_dict={model._images:batch_x, model.labels:batch_yhot})
        
        cost, pred = sess.run([model.cost, model.predictions], feed_dict={model._images:batch_x, model.labels:batch_yhot})
        costs.append(cost)
        print('Step: ', k, ' Cost: ', cost, ' LR: ', lr)
        print(pred[0])
        if(k % 100 is 0 and k > 0):
            plt.plot(costs)
            plt.show()
    

In [ ]:
plt.plot(costs)